In [1]:
// 1. Installation

//install libraries
import org.archive.archivespark._
import org.archive.archivespark.functions._
import org.archive.archivespark.specific.warc._

// data files - generic path from docker run -v
val cdxPath = "/data/arc_cdx/*.cdx"
val warcPath = "/data/warc"

import org.apache.spark.sql.{Row, SparkSession}

val session = spark.newSession

// collect all records

val r = ArchiveSpark.load(WarcSpec.fromFiles(cdxPath, warcPath))

In [2]:
val researchYear = "1996"

In [3]:
// 2. Count ALL objects from research Year
// 2.1 get data for text/html & HTTP 200 for each Year
val t1 = r.filter(r => r.timestamp.startsWith(researchYear))


In [4]:
// 3. generate basic data frame

val m = t1.map(f=> (f.originalUrl,f.digest,f.status,f.mime))
val df = session.createDataFrame(m).toDF("originalUrl","digest","status","mime").cache()

In [5]:
import org.apache.spark.sql.functions.count
import org.apache.spark.sql.functions._


In [7]:
df.filter(df("mime") === "text/html").select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                      38045|
+---------------------------+



In [8]:
df.select(countDistinct("originalUrl")).show()

+---------------------------+
|count(DISTINCT originalUrl)|
+---------------------------+
|                      72229|
+---------------------------+



In [16]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").orderBy(desc("URLs")).show(100,false)

+-------------------------+-----+
|mime                     |URLs |
+-------------------------+-----+
|text/html                |38045|
|image/gif                |21091|
|text/plain               |7951 |
|image/jpeg               |4105 |
|application/postscript   |179  |
|image/x-xbitmap          |145  |
|unk                      |134  |
|audio/x-wav              |113  |
|application/zip          |85   |
|application/octet-stream |75   |
|application/pdf          |70   |
|application/x-troff      |55   |
|application/x-dvi        |44   |
|application/x-tar        |21   |
|application/x-tex        |20   |
|application/x-msdownload |15   |
|video/mpeg               |14   |
|audio/x-pn-realaudio     |9    |
|application/rtf          |8    |
|application/x-excel      |8    |
|image/tiff               |7    |
|audio/basic              |4    |
|application/mac-binhex40 |3    |
|application/x-troff-man  |3    |
|video/quicktime          |3    |
|image/x-xpixmap          |2    |
|application/x

In [17]:
df.groupBy("mime").agg(countDistinct("originalUrl") as "URLs").count()

42